In [277]:
!/home/ubuntu/anaconda2/envs/py36/bin/pip install geocoder

    100% |████████████████████████████████| 92kB 954kB/s 
    100% |████████████████████████████████| 829kB 753kB/s 
  Running setup.py bdist_wheel for future ... - \ | done
  Stored in directory: /home/ubuntu/.cache/pip/wheels/c2/50/7c/0d83b4baac4f63ff7a765bd16390d2ab43c93587fac9d6017a
Successfully built future


In [278]:
# подгружаем все нужные пакеты
import pandas as pd
import numpy as np

# import pandas_profiling as pp


import itertools
import re
import os
import sys
import math
import string
import itertools as it
from transliterate import translit

from collections import defaultdict

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, classification_report, precision_score, recall_score, roc_curve
from sklearn.model_selection import train_test_split

# для встроенных картинок

# чуть покрасивше картинки:
pd.set_option('display.mpl_style', 'default')
figsize(12, 9)

import warnings
warnings.filterwarnings("ignore")

#plt.rcParams['figure.figsize'] = 10, 7.5
#plt.rcParams['axes.grid'] = True
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
# import seaborn as sns


import matplotlib as mpl
mpl.rcParams['font.family'] = 'Verdana' # Ubuntu

plt.rc('text', usetex=False)
plt.rc('font', family='serif')
plt.rc('font', weight='bold')
plt.rc('xtick', labelsize=14) 
plt.rc('ytick', labelsize=14)

# чтобы был русский шрифт
from matplotlib import rc
 
font = {'family': 'Verdana', #Droid Sans
        'weight': 'normal'}
rc('font', **font)
import pickle as pkl
import geojson
from matplotlib import path
from shapely import geometry
from matplotlib.collections import PatchCollection
from matplotlib.patches import Polygon
import vincenty

import geocoder
%matplotlib inline

## Regions polygons

In [2]:
regions = geojson.loads(open('../hackathon_mckinsey/moscow_regions_geojson.geojson').read())

In [3]:
region_polygons = {(region['properties']['ADMIN_LVL'], region['properties']['NAME']):
                   region['geometry']['coordinates']
                   for region in regions['features']}
region_polygons = {k:v[0][0]for k,v in region_polygons.items() if k[0] in ['8']}

In [4]:
region_paths = {k:path.Path(v) for k,v in region_polygons.items()}

### Load dist and origin 
delete nans add city field for outbound spots

In [63]:
df_text_origin = pd.read_csv('./text_origin.csv')
df_text_origin = df_text_origin[(df_text_origin.origin_latitude != 'null') & (df_text_origin.origin_longitude != 'null')]
df_text_origin.origin_latitude = df_text_origin.origin_latitude.astype(np.float64)
df_text_origin.origin_longitude = df_text_origin.origin_longitude.astype(np.float64)

In [64]:
df_text_dest = pd.read_csv('./text_dest.csv')
df_text_dest = df_text_dest[(df_text_dest.dest_latitude != 'null') & (df_text_dest.dest_longitude != 'null')]
df_text_dest.dest_latitude = df_text_dest.dest_latitude.astype(np.float64)
df_text_dest.dest_longitude = df_text_dest.dest_longitude.astype(np.float64)

### Taking only citites and throwing trashy things out

In [65]:
cities_dest = df_text_dest.dest_full_address.str.split(pat=',').apply(lambda x: x[-1])
cities_origin = df_text_origin.origin_full_address.str.split(pat=',').apply(lambda x: x[-1])

#cities_dest = [re.sub(pattern='Терминал\s*[A-ZА-Я]', flags=re.IGNORECASE, string=i, repl='').lower() for i in cities_dest]
#cities_origin = [re.sub(pattern='Терминал\s*[A-ZА-Я]', flags=re.IGNORECASE, string=i, repl='').lower() for i in cities_origin]

Append to DFs

In [74]:
df_text_dest['city'] = cities_dest
df_text_origin['city'] = cities_origin

In [ ]:
unique = [i for i in df_text_dest.city.unique() if not_external(string=i)]

In [68]:
def sub_string(string):
    if re.match(pattern='\s*.*Шереметьево.*', string=string, flags=re.IGNORECASE) is not None or re.match(pattern=r'\s*.*141425\s*.*', string=string, flags=re.IGNORECASE) is not None or re.match(pattern='\s*.*Sheremetyevo.*', string=string, flags=re.IGNORECASE):
        return 'Шереметьево'
    elif re.match(pattern='\s*.*Mosk[vw]{1}a.*', string=string, flags=re.IGNORECASE) is not None:
        return 'Москва'
    elif re.match(pattern='\s*.*Внуково\s*.*', string=string, flags=re.IGNORECASE) is not None or re.match(pattern=r'\s*.*142750\s*.*', string=string, flags=re.IGNORECASE) is not None or re.match(pattern='\s*.*119027\s*.*', string=string, flags=re.IGNORECASE) is not None:
        return 'Внуково'
    elif re.match(pattern='\s*.*Домодедово\s*.*', string=string, flags=re.IGNORECASE) is not None or re.match(pattern='\s*.*Domodedovo\s*.*', string=string, flags=re.IGNORECASE)is not None:
        return 'Домодедово'
    else:
        return re.sub(pattern=' ', repl='', string=string)

In [75]:
df_text_dest['city_cor'] = df_text_dest.city.apply(lambda x: sub_string(x))
df_text_origin['city_cor'] = df_text_origin.city.apply(lambda x: sub_string(x))

In [76]:
intersting_cities = df_text_dest.city_cor.value_counts()[df_text_dest.city_cor.value_counts() > 300]

In [77]:
df_text_dest = df_text_dest[df_text_dest.city_cor.isin(set(intersting_cities.index))]
df_text_origin = df_text_origin[df_text_origin.city_cor.isin(set(intersting_cities.index))]

In [80]:
%%time
for k,v in region_paths.items():
    df_text_origin[k[1]] = v.contains_points(df_text_origin[['origin_longitude', 'origin_latitude']])
    df_text_dest[k[1]] = v.contains_points(df_text_dest[['dest_longitude', 'dest_latitude']])
    print('\r{} done'.format(k[1]), end=' ')

район Некрасовка done CPU times: user 1min 32s, sys: 1.55 s, total: 1min 33s
Wall time: 1min 33s


In [82]:
regions_list = [k[1] for k in region_paths.keys()]

In [83]:
df_text_dest['region_dest'] = df_text_dest[regions_list].idxmax(axis=1)
df_text_origin['region_origin'] = df_text_origin[regions_list].idxmax(axis=1)

In [84]:
## delete dummy values of districts
for c in regions_list:
    del df_text_dest[c]
    del df_text_origin[c]

In [90]:
df_text_dest['district_final'] = df_text_dest.apply(lambda x: x.city_cor if x.region_dest == 'поселение Киевский' else x.region_dest, axis=1)
df_text_origin['district_final'] = df_text_origin.apply(lambda x: x.city_cor if x.region_origin == 'поселение Киевский' else x.region_origin, axis=1)

In [105]:
df_text_dest = df_text_dest[~df_text_dest.district_final.isin(['Россия', 'null', 'Russia', 'Пятницкоешоссе', 'Moscow', 'ВНИИССОК'])]
df_text_origin = df_text_origin[~df_text_origin.district_final.isin(['Россия', 'null', 'Russia', 'Пятницкоешоссе', 'Moscow', 'ВНИИССОК'])]

In [114]:
dest_by_district = df_text_dest.groupby(['hour_key','district_final']).agg(
    {'order_gk': 'count'}
)
origin_by_district = df_text_origin.groupby(['hour_key', 'district_final']).agg(
    {'order_gk': 'count'}
)

In [133]:
dest_by_district = dest_by_district.reset_index()
origin_by_district = origin_by_district.reset_index()

In [134]:
dest_by_district['share'] = dest_by_district.apply(lambda x: x['order_gk']/dest_by_district[
        (dest_by_district['hour_key'] == x['hour_key'])]['order_gk'].sum(), axis=1)
origin_by_district['share'] = origin_by_district.apply(lambda x: x['order_gk']/origin_by_district[
        (origin_by_district['hour_key'] == x['hour_key'])]['order_gk'].sum(), axis=1)

In [331]:
demand_offer_balance = dest_by_district.merge(origin_by_district, on=('hour_key', 'district_final'), suffixes=('supply', 'demand'))

In [260]:
mpol = geometry.MultiPolygon([geometry.Polygon(v) for v in region_polygons.values()])
polys = {k: geometry.Polygon(v) for k,v in region_polygons.items()}
polys_mpl = {k: Polygon(v) for k,v in region_polygons.items()}

centroids = {k[1]: v.centroid.coords[0] for k,v in polys.items()}

### Polygons for unindexed regions

In [315]:
gs = {i: geocoder.google(i) for i in unknown_yet}

gs_corr = {}
for k, g in gs.items():
    try:
        gs_corr[k] = g.current_result.latlng
    except:
        print('oops')

oops
oops
oops


In [366]:
clockwise = [(0,0), (1,0), (1,1), (0,1)]
def polygon_from_corners(corners):
    try:
        lst = []
        for x_id, y_id in clockwise:
            lst.append((corners[x_id][1], corners[y_id][0]))
        lst.append(lst[0])
        return path.Path(lst)
    except:
        print('oops')

In [367]:
unindexed_polygons = {}
for k,v in gs.items():
    try:
        unindexed_polygons[k] = polygon_from_corners(list(v.current_result.bbox.values()))
    except:
        print('oops')

oops
oops
oops


In [376]:
region_paths_add = region_paths
for k,v in unindexed_polygons.items():
    region_paths_add[('8',k)] = v

In [378]:
rides_coords = pd.read_csv('./for_lenya.csv')

In [382]:
rides_coords.columns #lat:55, lon:37

Index(['x_lat', 'x_lon', 'y_lat', 'y_lon'], dtype='object')

In [383]:
%%time
for k,v in region_paths_add.items():
    rides_coords[k[1] + '_origin'] = v.contains_points(rides_coords[['x_lon', 'x_lat']])
    rides_coords[k[1] + '_destination'] = v.contains_points(rides_coords[['y_lon', 'y_lat']])
    print('\r{} done'.format(k[1]), end=' ')

Бутово done CPU times: user 17.4 s, sys: 76 ms, total: 17.4 s
Wall time: 17.4 s


In [385]:
regions_list_add = [k[1] for k in region_paths_add.keys()]

In [386]:
rides_coords['region_origin'] = rides_coords[[i+'_origin' for i in regions_list_add]].idxmax(axis=1)
rides_coords['region_dest'] = rides_coords[[i+'_destination' for i in regions_list_add]].idxmax(axis=1)

In [388]:
## delete dummy values of districts
for c in regions_list_add:
    for addi in ['_origin', '_destination']:
        del rides_coords[c + addi]

In [391]:
rides_coords['region_origin'] = rides_coords.apply(lambda x: re.sub(string=x.region_origin, pattern='_origin', repl=''), axis=1)
rides_coords['region_dest'] = rides_coords.apply(lambda x: re.sub(string=x.region_dest, pattern='_destination', repl=''), axis=1)

In [400]:
rides_coords_pivot = pd.pivot_table(data=rides_coords, columns=('region_origin', 'region_dest'), aggfunc='count').reset_index()

### Probabilty estimates for riding from one region to another

In [405]:
rides_coords_pivot['share'] = rides_coords_pivot.apply(lambda x: x[0]/rides_coords_pivot[
        (rides_coords_pivot['region_origin'] == x['region_origin'])][0].sum(), axis=1)

In [321]:
gs_corr = {}
foar k, g in gs.items():
    try:
        gs_corr[k] = g.current_result.latlng[::-1]
    except:
        print('oops')

oops
oops
oops


In [322]:
for k,v in gs_corr.items():
    centroids[k] = v

In [332]:
demand_offer_balance_pip = demand_offer_balance[demand_offer_balance.district_final.isin(centroids.keys())]

In [335]:
demand_offer_balance_pip[demand_offer_balance_pip.district_final == 'Шереметьево']

,index,hour_key,district_final,order_gksupply,sharesupply,order_gkdemand,sharedemand
77,78,0,Шереметьево,291,0.017056,2685,0.100698
256,262,1,Шереметьево,322,0.029525,713,0.039993
433,442,2,Шереметьево,409,0.060253,600,0.052975
605,616,3,Шереметьево,924,0.145489,631,0.065633
780,792,4,Шереметьево,2124,0.263230,1166,0.110239
950,965,5,Шереметьево,3364,0.316136,983,0.075691
1125,1142,6,Шереметьево,3784,0.252115,926,0.052809
1300,1321,7,Шереметьево,2713,0.142101,639,0.025353
1478,1502,8,Шереметьево,1405,0.074893,655,0.022566
1660,1686,9,Шереметьево,956,0.063687,602,0.024616


In [338]:
len(centroids.keys()) ** 2

36864

### Pairwise distance between districts

In [324]:
vinc_pairwise = {}
for (k1, v1), (k2,v2) in itertools.product(centroids.items(), repeat=2):
    vinc_pairwise[(k1,k2)] = vincenty.vincenty(miles=False, point1=v1, point2=v2)

In [325]:
vinc_pairwise

{('поселение Киевский', 'поселение Киевский'): 0.0,
 ('поселение Киевский', 'район Филёвский Парк'): 72.528747,
 ('поселение Киевский', 'поселение Новофёдоровское'): 7.131186,
 ('поселение Киевский', 'район Кунцево'): 32.921757,
 ('поселение Киевский', 'поселение Роговское'): 23.627337,
 ('поселение Киевский', 'район Зюзино'): 80.066856,
 ('поселение Киевский', 'поселение Вороновское'): 26.556865,
 ('поселение Киевский', 'поселение Михайлово-Ярцевское'): 26.816236,
 ('поселение Киевский', 'поселение Марушкинское'): 35.994644,
 ('поселение Киевский', 'поселение Первомайское'): 35.637377,
 ('поселение Киевский', 'район Матушкино'): 64.343472,
 ('поселение Киевский', 'район Внуково'): 45.551334,
 ('поселение Киевский', 'район Савёлки'): 65.27082,
 ('поселение Киевский', 'поселение Внуковское'): 47.947726,
 ('поселение Киевский', 'район Силино'): 62.245617,
 ('поселение Киевский', 'поселение Кокошкино'): 35.147997,
 ('поселение Киевский', 'район Крюково'): 60.057028,
 ('поселение Киевский'

In [407]:
unknown_yet = set(demand_offer_balance.district_final.unique()) - set(centroids)

## Sum up - what do we have now

#### Pairwise distances between districts

In [409]:
vinc_pairwise

{('поселение Киевский', 'поселение Киевский'): 0.0,
 ('поселение Киевский', 'район Филёвский Парк'): 72.528747,
 ('поселение Киевский', 'поселение Новофёдоровское'): 7.131186,
 ('поселение Киевский', 'район Кунцево'): 32.921757,
 ('поселение Киевский', 'поселение Роговское'): 23.627337,
 ('поселение Киевский', 'район Зюзино'): 80.066856,
 ('поселение Киевский', 'поселение Вороновское'): 26.556865,
 ('поселение Киевский', 'поселение Михайлово-Ярцевское'): 26.816236,
 ('поселение Киевский', 'поселение Марушкинское'): 35.994644,
 ('поселение Киевский', 'поселение Первомайское'): 35.637377,
 ('поселение Киевский', 'район Матушкино'): 64.343472,
 ('поселение Киевский', 'район Внуково'): 45.551334,
 ('поселение Киевский', 'район Савёлки'): 65.27082,
 ('поселение Киевский', 'поселение Внуковское'): 47.947726,
 ('поселение Киевский', 'район Силино'): 62.245617,
 ('поселение Киевский', 'поселение Кокошкино'): 35.147997,
 ('поселение Киевский', 'район Крюково'): 60.057028,
 ('поселение Киевский'

#### Probability estimates of supply and demand in district given an hour

In [410]:
demand_offer_balance_pip.head()

,index,hour_key,district_final,order_gksupply,sharesupply,order_gkdemand,sharedemand
0,0,0,Академический район,84,0.004924,89,0.003338
1,1,0,Алексеевский район,95,0.005568,74,0.002775
2,2,0,Алтуфьевский район,12,0.000703,12,0.000450
3,3,0,Апрелевка,12,0.000703,25,0.000938
4,4,0,Архангельское,11,0.000645,11,0.000413


#### Probabilty estimates of order from district $i$ to district $j$

In [411]:
rides_coords_pivot.head()

,level_0,region_origin,region_dest,0,share
0,x_lat,Академический район,Академический район,416,0.124700
1,x_lat,Академический район,Басманный район,9,0.002698
2,x_lat,Академический район,Бутырский район,3,0.000899
3,x_lat,Академический район,Войковский район,1,0.000300
4,x_lat,Академический район,Гагаринский район,26,0.007794


## What we're going to need further is cost ride estimate, cost of moving from district to district

In [475]:
## price rate is about 10rub per km
def cost(rho):
    return 50 + 10*rho
def move_cost(rho):
    return 20 + 4*rho

### Q-learning like reward for predicting best available strategy
Driver strategy is a sequence of districts that he visits and gained amount of money, which is a reward, is:
$$
    r = \sum_{i=0}^n \gamma^i \cdot p(district_i, district_{i+1}|time) \cdot cost(\rho(district_i, district_{i+1})); \\ n=3,4,5
$$
We are going to do a brute force traversial of the tree of scanarios to the depth of 3 for a given region and a given time

In [422]:
rides_coords_pivot.columns

Index(['level_0', 'region_origin', 'region_dest', 0, 'share'], dtype='object')

In [423]:
demand_offer_balance_pip.columns

Index(['index', 'hour_key', 'district_final', 'order_gksupply', 'sharesupply',
       'order_gkdemand', 'sharedemand'],
      dtype='object')

In [426]:
demand_offer_balance_pip[(demand_offer_balance_pip.district_final == start_district)
                         & (demand_offer_balance_pip.hour_key == start_hour)]

,index,hour_key,district_final,order_gksupply,sharesupply,order_gkdemand,sharedemand
534,543,3,Академический район,16,0.002519,41,0.004265


In [554]:
def get_probs(district, hour, df_probs_across_districts):
    probs = df_probs_across_districts[(df_probs_across_districts.region_origin == district)
                   & (df_probs_across_districts.region_dest != start_district)].share/df_probs_across_districts[(df_probs_across_districts.region_origin == district)
                   & (df_probs_across_districts.region_dest != district)].share.sum()
    districts = df_probs_across_districts[(df_probs_across_districts.region_origin == district)
                   & (df_probs_across_districts.region_dest != start_district)].region_dest
    return pd.concat([probs, districts], axis=1)

def get_cost(district1, district2, dists):
    return cost(dists[(district1, district2)])

def w_cost_from_districts_sequence(districts, dists, gama=0.75):
    
    cost = 0
    for num, i in enumerate(districts[:-1]):
        prob = get_probs(i, hour=start_hour,df_probs_across_districts=rides_coords_pivot)
        cost += gama**num * prob[prob.region_dest==districts[num+1]].share.mean() * get_cost(i, districts[num+1], dists)
    return cost

In [551]:
## given time and district
start_district = 'район Ясенево'
start_hour = 3

gammaa = 0.8
end_depth = 2

In [515]:
districts = rides_coords_pivot.region_dest.unique()

index = pd.MultiIndex.from_product([districts for i in range(end_depth)],
                                   names = ['depth_{}'.format(i) for i in range(end_depth)])

In [552]:
cum_df = pd.DataFrame(index=index).reset_index()
cum_df = cum_df[cum_df.depth_0 == 'район Ясенево']

In [555]:
cum_df['cost'] = cum_df.apply(lambda x: w_cost_from_districts_sequence(
        districts=x[['depth_{}'.format(i) for i in range(end_depth)]], dists=vinc_pairwise),
        axis=1
)

In [563]:
cum_df.dropna().iloc[cum_df.dropna().cost.argsort()[::-1]]

,depth_0,depth_1,cost
11673,район Ясенево,Москва,3.174103
11732,район Ясенево,район Тёплый Стан,2.237351
11683,район Ясенево,Тверской район,1.864196
11699,район Ясенево,район Коньково,1.864090
11744,район Ясенево,район Якиманка,1.787590
11687,район Ясенево,поселение Сосенское,1.369384
11731,район Ясенево,район Тропарёво-Никулино,1.338244
11698,район Ясенево,район Зюзино,1.258184
11680,район Ясенево,Пресненский район,1.243240
11696,район Ясенево,район Замоскворечье,1.106613
